# Random Forest
Hot nuclei top top-attention quintile.  
Run on Windows / Precision.
Train on 5-class cancer classification.  
Input is the nucleus+RBC rollup, training set, hot nuclei only.    
Run 5-fold cross validation.  
Print the mean accuracy. 

The Image.csv is a copy of the original.  
The RBC_rollup.csv is a copy of the original.  
The Nucleus_rollup.csv summarizes only the high-temperature nuclei.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
#import tensorflow as tf
#tf.config.list_physical_devices('GPU')

2022-06-16 17:13:02.663446
Python 3.9.6
sklearn 1.1.1


In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models
from CellProfiler_Util import CP_Util
from RandomForestUtil import RF_Util

In [3]:
BASE_DIR='D:\\Adjeroh\\Naved\\CP_80K_HotQuint\\'  # append Output0/ etc
CLASSES=range(0,6)  # use all 6 classes
CLASSES=range(1,6)  # use just 5 classes
MODELS_DIR='D:\\Adjeroh\\Naved\\CP_80K\\models\\RandomForest.23'

In [4]:
print(datetime.datetime.now())
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        print(datetime.datetime.now())
        print("Process class",i)
        FULL_PATH=BASE_DIR+'Output'+str(i)+'/'
        cp = CP_Util(FULL_PATH)
        cp.train_test_split()
        Xnuc = cp.get_nucleus_rollup()
        set_prefix('Nuc_')
        Xnuc.rename(get_prefix,axis='columns',inplace=True)
        #Xrbc = cp.get_RBC_rollup()
        #set_prefix('Rbc_')
        #Xrbc.rename(get_prefix,axis='columns',inplace=True)
        #Xi = Xnuc.join(Xrbc,how='outer')
        Xi = Xnuc
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-16 17:13:03.913762
2022-06-16 17:13:03.913762
Process class 1
Train: 17 participants, 28 WSI, 10925 patches.
Test: 4 participants, 5 WSI, 1977 patches.
2022-06-16 17:13:21.595815
Process class 2
Train: 14 participants, 30 WSI, 11138 patches.
Test: 3 participants, 7 WSI, 2812 patches.
2022-06-16 17:13:40.275869
Process class 3
Train: 11 participants, 13 WSI, 5006 patches.
Test: 3 participants, 4 WSI, 1358 patches.
2022-06-16 17:13:49.332243
Process class 4
Train: 6 participants, 7 WSI, 2809 patches.
Test: 1 participants, 1 WSI, 388 patches.
2022-06-16 17:13:54.048633
Process class 5
Train: 2 participants, 3 WSI, 1195 patches.
Test: 1 participants, 1 WSI, 396 patches.


,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,Nuc_AreaShape_Area_std,Nuc_AreaShape_Area_min,Nuc_AreaShape_Area_25%,Nuc_AreaShape_Area_50%,Nuc_AreaShape_Area_75%,Nuc_AreaShape_Area_max,Nuc_AreaShape_BoundingBoxArea_count,Nuc_AreaShape_BoundingBoxArea_mean,...,Nuc_Texture_Variance_Hematoxylin_7_02_256_75%,Nuc_Texture_Variance_Hematoxylin_7_02_256_max,Nuc_Texture_Variance_Hematoxylin_7_03_256_count,Nuc_Texture_Variance_Hematoxylin_7_03_256_mean,Nuc_Texture_Variance_Hematoxylin_7_03_256_std,Nuc_Texture_Variance_Hematoxylin_7_03_256_min,Nuc_Texture_Variance_Hematoxylin_7_03_256_25%,Nuc_Texture_Variance_Hematoxylin_7_03_256_50%,Nuc_Texture_Variance_Hematoxylin_7_03_256_75%,Nuc_Texture_Variance_Hematoxylin_7_03_256_max
PatchNumber,,,,,,,,,,,,,,,,,,,,,
406,11.0,500.545455,309.300942,200.0,281.50,354.0,760.00,1020.0,11.0,781.727273,...,1277.099383,1827.079462,11.0,913.517423,504.979930,239.734047,599.500761,713.686310,1260.287418,1878.422613
407,15.0,428.200000,237.755937,195.0,250.00,359.0,516.00,1091.0,15.0,668.266667,...,1197.326142,1759.446234,15.0,988.752237,259.969754,477.783754,884.728441,1049.426461,1107.627554,1479.370708
408,1.0,382.000000,0.000000,382.0,382.00,382.0,382.00,382.0,1.0,552.000000,...,1813.652730,1813.652730,1.0,2227.185610,0.000000,2227.185610,2227.185610,2227.185610,2227.185610,2227.185610
409,11.0,446.454545,221.708531,221.0,277.00,398.0,500.50,949.0,11.0,688.272727,...,1038.282898,1284.991669,11.0,868.611696,442.121503,250.826972,540.682762,839.909439,1111.565076,1637.398754
411,12.0,388.666667,151.452563,179.0,263.00,364.5,517.25,599.0,12.0,539.833333,...,851.043891,1388.155312,12.0,682.561278,360.995923,189.819252,429.751089,559.106503,970.406474,1374.917516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,4.0,328.750000,99.684084,241.0,243.25,327.0,412.50,420.0,4.0,634.500000,...,773.052956,888.751162,4.0,538.876642,228.881660,287.399558,386.900587,534.546618,686.522673,799.013775
1587,1.0,323.000000,0.000000,323.0,323.00,323.0,323.00,323.0,1.0,416.000000,...,5002.663831,5002.663831,1.0,4643.175378,0.000000,4643.175378,4643.175378,4643.175378,4643.175378,4643.175378
1588,5.0,392.000000,102.022056,266.0,320.00,410.0,437.00,527.0,5.0,566.800000,...,3084.913423,3144.417339,5.0,2460.085164,752.282885,1511.709857,1943.244606,2492.375206,2990.138028,3362.958125


In [5]:
print(datetime.datetime.now())
print("Shuffle...")
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-16 17:13:56.704575
Shuffle...


In [6]:
print(datetime.datetime.now())
print('Cross-validation...')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
cv_scores = rf1.cross_validation()
print(cv_scores)
print('mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-16 17:13:57.177918
Cross-validation...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 2.2min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.2min remaining:    0.0s


[CV] END .................................................... total time= 2.2min
[CV] END .................................................... total time= 2.2min
[CV] END .................................................... total time= 2.2min
[CV] END .................................................... total time= 2.2min
[0.58278963 0.58243696 0.57979192 0.57742504 0.57336861]
mean 0.5792 +/- 0.0035


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 11.0min finished


In [7]:
print(datetime.datetime.now())
joblib.dump(rf1.get_model(),MODELS_DIR)

2022-06-16 17:24:55.092300


['D:\\Adjeroh\\Naved\\CP_80K\\models\\RandomForest.23']